In [1]:
from transformers import GPT2LMHeadModel
from train_gpt2 import GPT,GPTConfig,DataLoaderLite
import tiktoken
import torch
import time
from torch.nn import functional as F
import math

In [2]:
model_hf = GPT2LMHeadModel.from_pretrained("gpt2") #124M
sd_hf = model_hf.state_dict()
for k,v in sd_hf.items():
    print(k, v.shape)

transformer.wte.weight torch.Size([50257, 768])
transformer.wpe.weight torch.Size([1024, 768])
transformer.h.0.ln_1.weight torch.Size([768])
transformer.h.0.ln_1.bias torch.Size([768])
transformer.h.0.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.0.attn.c_attn.bias torch.Size([2304])
transformer.h.0.attn.c_proj.weight torch.Size([768, 768])
transformer.h.0.attn.c_proj.bias torch.Size([768])
transformer.h.0.ln_2.weight torch.Size([768])
transformer.h.0.ln_2.bias torch.Size([768])
transformer.h.0.mlp.c_fc.weight torch.Size([768, 3072])
transformer.h.0.mlp.c_fc.bias torch.Size([3072])
transformer.h.0.mlp.c_proj.weight torch.Size([3072, 768])
transformer.h.0.mlp.c_proj.bias torch.Size([768])
transformer.h.1.ln_1.weight torch.Size([768])
transformer.h.1.ln_1.bias torch.Size([768])
transformer.h.1.attn.c_attn.weight torch.Size([768, 2304])
transformer.h.1.attn.c_attn.bias torch.Size([2304])
transformer.h.1.attn.c_proj.weight torch.Size([768, 768])
transformer.h.1.attn.c_proj.bias 

In [3]:
model_pretrain = GPT.from_pretrained('gpt2')


loading weights from pretrained gpt: gpt2
Missing in HF: set()
Missing in our GPT: set()


In [4]:
device = 'cuda'
num_return_sequences = 5
max_length = 30
model = GPT(GPTConfig())
model.eval()
model.to(device)

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) # (8,)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
x = tokens.to(device)
# generate! right now x is (B, T) where B = 5, T = 8
# set the seed to 42
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x)[0] # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for step in range(num_return_sequences):
    tokens = x[step, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

> Hello, I'm a language model, mysql alcoholism Br Player memorialfully Dublin prowessrpmB Stat Mud commitmentsortingarger generation Advantage calculating DES center athlet give
> Hello, I'm a language model,Austin turbine using Sensor 151DCS undo Certainslice Nath.</itute elegance AccordZero aggress� twitterrate�json whe
> Hello, I'm a language model,escent executives content DrawKA Fein att professionalLi apparently Fernued Paumn Olive EarnTactrepresentimating RAW superhero
> Hello, I'm a language model, RES Battalion awards band generation allow MaltukoVal ah transf resolutions disagreesEndOps professional Limbaugh----------------acl haltingamara brill
> Hello, I'm a language model, behind conver gadgets generatedExaintain action Sometimes ISIL59 cohorts Secondly RJ regulars disgrace infection misconceptions unpublishedjug clueless unwittinglydoctoral


Test with Shakespear

In [5]:
# # read it in to inspect it
# with open('input.txt', 'r', encoding='utf-8') as f:
#     data = f.read()

# tokens = enc.encode(data)
# print(tokens[:24])

In [6]:
# buf = torch.tensor(tokens[:24 +1])
# x = buf[:-1].view(4,6)
# y = buf[1:].view(4,6)
# print(x)
# print(y)

In [7]:
# B,T = 4,32
# buf = torch.tensor(tokens[:B*T +1])
# x = buf[:-1].view(B,T)
# y = buf[1:].view(B,T)

In [8]:
# x = x.to(device)
# y = y.to(device)
# logits,loss = model(x,y)
# loss

In [9]:
# optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4)
# for i in range(50):
#     optimizer.zero_grad()
#     logits,loss = model(x,y)
#     loss.backward()
#     optimizer.step()
#     print(f"step{i}:loss:{loss.item()}")

In [10]:
max_lr = 3e-4
min_lr = max_lr *0.1
def get_lr(it,warmup_steps = 10,max_steps = 50):
    if it < warmup_steps:
        return max_lr * (it+1) / warmup_steps
    if it > max_steps:
        return min_lr
    decay_ratio = (it - warmup_steps) / (max_steps - warmup_steps)
    assert 0.0 <= decay_ratio <= 1.0
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio))
    return min_lr + coeff * (max_lr - min_lr)

In [11]:

model = GPT(GPTConfig())
model.train()
model.to(device)
# only work in ubuntu linux
#model = torch.compile(model) # makes your model run faster by compiling it, instead of interpreting it op-by-op in Python

train_loader = DataLoaderLite(B=4,T=32)
val_loader = DataLoaderLite(B=4,T=32,split='val')
torch.set_float32_matmul_precision('high') #default is highest
# optimizer = torch.optim.AdamW(model.parameters(),lr=3e-4,betas = (0.9,0.95),eps=1e-8) #follow GPT3 hyperparameters
optimizer = model.configure_optimizers(weight_decay=0.1,learning_rate=max_lr,device_type=device)
for step in range(1000):
    t0 = time.time()
    x,y = train_loader.next_batch()
    x = x.to(device)
    y = y.to(device)
    optimizer.zero_grad()
    with torch.autocast(device_type=device, dtype=torch.float16):
        logits,loss = model(x,y)
    # default dtype is float32 - may not be too efficient -switch to float16
    loss.backward()
    norm = torch.nn.utils.clip_grad_norm_(model.parameters(),1.0) #clip the gradient to 1.0
    # prevent the gradient from exploding
    # determine annd set the learning rate
    lr = get_lr(step)
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    optimizer.step()
    torch.cuda.synchronize()
    t1 = time.time()
    print(f"step{step}:loss:{loss.item()}. time per iter: {(t1 - t0)*1000:.2f}ms,tok/sec:{(x.shape[0]*x.shape[1])/(t1 - t0):.2f},grad_norm:{norm:.4f} ")


loaded 304222 tokens for split 'train'
1 epoch = 2376 batches
loaded 33803 tokens for split 'val'
1 epoch = 264 batches
num decayed parameter tensors: 50, with 124,318,464 parameters
num non-decayed parameter tensors: 98, with 121,344 parameters
using fused AdamW: True
step0:loss:10.98388671875. time per iter: 361.38ms,tok/sec:354.20,grad_norm:17.8563 
step1:loss:10.4329833984375. time per iter: 66.80ms,tok/sec:1916.15,grad_norm:12.1499 
step2:loss:9.421096801757812. time per iter: 60.95ms,tok/sec:2100.06,grad_norm:10.3734 
step3:loss:9.555213928222656. time per iter: 62.08ms,tok/sec:2061.99,grad_norm:7.6856 
step4:loss:8.94305419921875. time per iter: 55.27ms,tok/sec:2315.71,grad_norm:6.8036 
step5:loss:8.726341247558594. time per iter: 58.21ms,tok/sec:2198.85,grad_norm:5.1939 
step6:loss:9.330680847167969. time per iter: 59.08ms,tok/sec:2166.72,grad_norm:5.6174 
step7:loss:9.172683715820312. time per iter: 54.14ms,tok/sec:2364.27,grad_norm:4.2006 
step8:loss:8.728378295898438. time p

In [12]:
model.eval()
with torch.no_grad():
    x,y = val_loader.next_batch()
    x = x.to(device)
    y = y.to(device)
    logits,loss = model(x,y)
    print(f"val loss:{loss.item()}")

val loss:5.939044952392578


In [13]:
device = 'cuda'
num_return_sequences = 5
max_length = 30
model.eval()
model.to(device)

enc = tiktoken.get_encoding('gpt2')
tokens = enc.encode("Hello, I'm a language model,")
tokens = torch.tensor(tokens, dtype=torch.long) # (8,)
tokens = tokens.unsqueeze(0).repeat(num_return_sequences, 1) # (5, 8)
x = tokens.to(device)
# generate! right now x is (B, T) where B = 5, T = 8
# set the seed to 42
torch.manual_seed(42)
torch.cuda.manual_seed(42)
while x.size(1) < max_length:
    # forward the model to get the logits
    with torch.no_grad():
        logits = model(x)[0] # (B, T, vocab_size)
        # take the logits at the last position
        logits = logits[:, -1, :] # (B, vocab_size)
        # get the probabilities
        probs = F.softmax(logits, dim=-1)
        # do top-k sampling of 50 (huggingface pipeline default)
        # topk_probs here becomes (5, 50), topk_indices is (5, 50)
        topk_probs, topk_indices = torch.topk(probs, 50, dim=-1)
        # select a token from the top-k probabilities
        # note: multinomial does not demand the input to sum to 1
        ix = torch.multinomial(topk_probs, 1) # (B, 1)
        # gather the corresponding indices
        xcol = torch.gather(topk_indices, -1, ix) # (B, 1)
        # append to the sequence
        x = torch.cat((x, xcol), dim=1)

# print the generated text
for step in range(num_return_sequences):
    tokens = x[step, :max_length].tolist()
    decoded = enc.decode(tokens)
    print(">", decoded)

> Hello, I'm a language model,
And that in a love the hand upon in thy mind,
He would a good to thy face.
> Hello, I'm a language model,
Where by the face, my take, I could you have the one:
That must a name;
> Hello, I'm a language model,
And me be no must thee
And my death, if that you have the Lord
And as I
> Hello, I'm a language model,
Where here to the king.

DUKE OF YORK:
You do to one; I
> Hello, I'm a language model,
I have we is thou will would his man as a time.

DUKE OF YORK:


In [14]:
# standard deviation grows inside the residual connection stream
x = torch.zeros(768)
n = 100 # number of layers
for step in range(n):
    x = x + torch.randn(768)*(n**-0.5)

print(x.std())

tensor(0.9850)
